In [ ]:
import sys
import os
import boto3
import rasterio

import pandas as pd
import geopandas as gpd

from rasterio.merge import merge

# Import raster helpers
sys.path.insert(0, "../../../../GOSTrocks/src")

import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

In [ ]:
in_folder = r'C:\WBG\Work\Projects\Novel_Urban\COG_DATA'
out_folder = r'C:\WBG\Work\Projects\Novel_Urban\COG_RESULTS'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

iso3 = 'COG'
pop_file = os.path.join(in_folder, 'DOU', 'cog_cpo20_WB.tif')
adm2 = dMisc.get_geoboundaries(iso3, 'ADM2')
#adm3 = dMisc.get_geoboundaries(iso3, 'ADM3')

In [ ]:
def summarize_urban(adm, pop_layer, urban_layer):
    """ Summarize urban population for each administrative unit
    Parameters
    ----------
    adm : geoDataFrame
        administrative unit boundary
    pop_layer : rasterio dataset
        population layer
    urban_layer : rasterio dataset
        urban layer
    """
        
    # Read raster data
    pop_data = pop_layer.read(1)
    urban_data = urban_layer.read(1)
    urban_data = (urban_data > 0) * 1
    
    urban_pop = pop_data * urban_data

    with rMisc.create_rasterio_inmemory(urban_layer.profile, urban_pop) as urban_pop_ds:
        res = rMisc.zonalStats(adm, urban_pop_ds, minVal=0)
        res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
    return(res['SUM'])

    

In [ ]:
# calculate urban population numbers
urban_res = adm2.copy()
pop_layer = rasterio.open(pop_file)
if adm2.crs != pop_layer.crs:
    adm2 = adm2.to_crs(pop_layer.crs)

res = rMisc.zonalStats(adm2, pop_layer, minVal=0)
res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
urban_res['total_pop'] = res['SUM']

for urb_file, lbl in [
        [os.path.join(in_folder, 'DOU', 'cog_cpo20_WB_urban.tif'), 'DoU_urban'],
        [os.path.join(in_folder, 'DOU', 'cog_cpo20_WB_urban_hd.tif'), 'DoU_urban_hd'],
        [os.path.join(in_folder, 'DB', 'cog_cpo20_WBd10b5000_cc.tif'), 'DB_cc'],
        [os.path.join(in_folder, 'DB', 'cog_cpo20_WBd10b5000_co.tif'), 'DB_co'],
        [os.path.join(in_folder, 'DB', 'cog_cpo20_WBd10b5000_ur.tif'), 'DB_ur'],
    ]:
    tPrint(f"Processing {lbl}")
    urban_res[lbl] = summarize_urban(adm2, pop_layer, rasterio.open(urb_file))
    

In [ ]:
pd.DataFrame(urban_res.drop(columns=['geometry'])).to_csv(os.path.join(out_folder, 'adm2_urban_pop.csv'), index=False)
urban_res.to_file(os.path.join(out_folder, 'adm2_urban_pop.geojson'), driver='GeoJSON')